# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [2]:
# Basic pandas and numpy
import pandas as pd
import numpy as np
 
# Pandas defaults
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

In [3]:
# Defining a hypothesis checker function:
def hyp_checker(pvalue):
    alpha = float(input("What's your alpha?"))
    if type(alpha) != float:
        print('Insert a correct alpha')
        return hyp_checker(result)
    else:
        if pvalue <= alpha:
            return "Our p value ({}) is smaller than our alpha({}). H0 is rejected".format(pvalue, alpha)
        else:
            return "Our p value ({}) is bigger than our alpha({}). Fail to reject H0".format(pvalue, alpha)

# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [4]:
# Run this code:
pokemon = pd.read_csv('../pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [5]:
# Your code here:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [6]:
pokemon.Legendary.value_counts()

False    735
True      65
Name: Legendary, dtype: int64

Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [7]:
poke_stats = pokemon[['Total', 'Legendary']].groupby('Legendary', as_index=True).agg({'Total':['mean','std']})
# Flattening columns:
poke_stats.columns = poke_stats.columns.map('|'.join).str.strip('|')
poke_stats

,Total|mean,Total|std
Legendary,,
False,417.213605,106.760417
True,637.384615,60.937389


The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [8]:
# H0 Total(legend) = Total(ordin)
# H1 Total(legend) != Total(ordin)
# Samples not matched. Unequal variances. So we use Welch's t-test.

In [9]:
from scipy.stats import ttest_ind
legend = pokemon.Total.loc[pokemon.Legendary == True]
ordin = pokemon.Total.loc[pokemon.Legendary == False]
result = ttest_ind(legend, ordin, equal_var=False)
result

Ttest_indResult(statistic=25.8335743895517, pvalue=9.357954335957446e-47)

What do you conclude from this test? Write your conclusions below.

In [10]:
hyp_checker(result[1])

What's your alpha?0.05


'Our p value (9.357954335957446e-47) is smaller than our alpha(0.05). H0 is rejected'

**Legendary and Not Legendary pokemons points differ**

How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [41]:
pokemon['Type 1'].value_counts()

Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Electric     44
Rock         44
Ghost        32
Ground       32
Dragon       32
Dark         31
Poison       28
Fighting     27
Steel        27
Ice          24
Fairy        17
Flying        4
Name: Type 1, dtype: int64

Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [63]:
poke_stats = pokemon.groupby(pokemon["Type 1"] == 'Water').Total.agg(['mean', 'std'])
poke_stats

,mean,std
Type 1,,
False,435.859012,121.091682
True,430.455357,113.188266


Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [ ]:
# H0 Total(water) = Total(not_water)
# H1 Total(water) != Total(not_water)
# Samples not matched. Equal variances. So we use Welch's t-test.

In [75]:
water = pokemon.Total.loc[pokemon['Type 1'] == 'Water']
not_water = pokemon.Total.loc[pokemon['Type 1'] != 'Water']
result = ttest_ind(water, not_water, equal_var=True)
result

Ttest_indResult(statistic=-0.4418547448849676, pvalue=0.6587140317488793)

Write your conclusion below.

In [78]:
hyp_checker(result[1])

What's your alpha?0.05


'Our p value (0.6587140317488793) is bigger than our alpha(0.05). Fail to reject H0'

**We can't conclude that Water type and not water type pokemons have equal points**

# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [ ]:
# H0 Defense = Attack
# H1 Defense != Attack

In [80]:
pokemon.columns

Index(['#', 'Name', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary'],
      dtype='object')

In [81]:
from scipy.stats import ttest_rel
attack = pokemon.Attack
defense = pokemon.Defense
result = ttest_rel(attack, defense)
result

Ttest_relResult(statistic=4.325566393330478, pvalue=1.7140303479358558e-05)

Describe the results of the test in the cell below.

In [82]:
hyp_checker(result[1])

What's your alpha?0.05


'Our p value (1.7140303479358558e-05) is smaller than our alpha(0.05). H0 is rejected'

**Defense and attack points differ**

We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [83]:
pokemon.columns

Index(['#', 'Name', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary'],
      dtype='object')

In [ ]:
# H0 Sp.Defense = Sp.Attack
# H1 Sp.Defense != Sp.Attack

In [85]:
from scipy.stats import ttest_rel
attack = pokemon['Sp. Atk']
defense = pokemon['Sp. Def']
result = ttest_rel(attack, defense)
result

Ttest_relResult(statistic=0.853986188453353, pvalue=0.3933685997548122)

Describe the results of the test in the cell below.

In [86]:
hyp_checker(result[1])

What's your alpha?0.05


'Our p value (0.3933685997548122) is bigger than our alpha(0.05). Fail to reject H0'

**We can't conclude that the points are the same.**

As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [ ]:
# H0 Sp.Defense - Sp.Attack = 0
# H1 Sp.Defense - Sp.Attack != 0

In [88]:
from scipy.stats import ttest_1samp
diff = defense - attack
result = ttest_1samp(diff, 0)
result

Ttest_1sampResult(statistic=-0.853986188453353, pvalue=0.3933685997548122)

**Same results**

# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [90]:
conting = pd.crosstab(pokemon['Legendary'], pokemon['Type 1'] == "Water")
conting

Type 1,False,True
Legendary,,
False,627,108
True,61,4


Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [93]:
from scipy.stats import chi2_contingency
result = chi2_contingency(conting)
result

(2.9429200762850503, 0.08625467249550949, 1, array([[632.1, 102.9],
        [ 55.9,   9.1]]))

Based on a 95% confidence, should we reject the null hypothesis?

In [95]:
hyp_checker(result[1])

What's your alpha?0.05


'Our p value (0.08625467249550949) is bigger than our alpha(0.05). Fail to reject H0'